# Loading the Data

In [1]:
import json
import re
from os.path import join
import os

import pandas as pd
import numpy as np
from tqdm import tqdm
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import optuna
import shap
import cfbd
from sklearn.metrics import log_loss, accuracy_score, classification_report, confusion_matrix, roc_auc_score
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

with open('../config.json') as f:
    configuration = cfbd.Configuration(
        access_token = json.load(f)['CFBD_API_KEY']
    )

In [2]:
YEARS = range(2013, 2025)
DATA_DIR = '../data'

In [3]:
def convert_to_snake_case(cols):
    cols_new = []
    for c in cols:
        s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', c)
        cols_new.append(re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower().replace('t_ds', 'tds'))
    return cols_new

In [4]:
# Load weather data
all_dfs = []

weather_dir = join(DATA_DIR, 'weather')
if not os.path.exists(weather_dir):
    os.makedirs(weather_dir)

for year in YEARS:
    file_path = join(weather_dir, f'{year}.parquet')
    if os.path.exists(file_path):
        print(f'Reading {file_path} from cached data')
        tmp = pd.read_parquet(file_path)
    else:
        print(f'Fetching {year} weather data from CFBD API')
        with cfbd.ApiClient(configuration) as api_client:
            api_instance = cfbd.GamesApi(api_client)
            data = api_instance.get_weather(year=year)
        tmp = pd.DataFrame([val.to_dict() for val in data])
        tmp.columns = convert_to_snake_case(tmp.columns)
        tmp['season_type'] = tmp['season_type'].apply(lambda x: x.value)
        tmp.to_parquet(file_path)
    all_dfs.append(tmp)
del tmp

weather = pd.concat(all_dfs, ignore_index=True).reset_index(drop=True)
weather.head()

Reading ../data/weather/2013.parquet from cached data
Reading ../data/weather/2014.parquet from cached data
Reading ../data/weather/2015.parquet from cached data
Reading ../data/weather/2016.parquet from cached data
Reading ../data/weather/2017.parquet from cached data
Reading ../data/weather/2018.parquet from cached data
Reading ../data/weather/2019.parquet from cached data
Reading ../data/weather/2020.parquet from cached data
Reading ../data/weather/2021.parquet from cached data
Reading ../data/weather/2022.parquet from cached data
Reading ../data/weather/2023.parquet from cached data
Reading ../data/weather/2024.parquet from cached data


,id,season,week,season_type,start_time,game_indoors,home_team,home_conference,away_team,away_conference,venue_id,venue,temperature,dew_point,humidity,precipitation,snowfall,wind_direction,wind_speed,pressure,weather_condition_code,weather_condition
0,333272229,2013,13,regular,2013-11-23 23:00:00+00:00,False,Florida International,Conference USA,Marshall,Conference USA,218,FIU Stadium,78.1,70.0,76.0,0.0,0,70.0,5.8,1015.9,NaN,None
1,333062229,2013,10,regular,2013-11-02 22:00:00+00:00,False,Florida International,Conference USA,East Carolina,Conference USA,218,FIU Stadium,84.9,65.8,53.0,0.0,0,260.0,7.0,1010.5,NaN,None
2,332992229,2013,9,regular,2013-10-26 22:00:00+00:00,False,Florida International,Conference USA,Louisiana Tech,Conference USA,218,FIU Stadium,79.0,61.0,54.0,0.0,0,60.0,11.4,1018.2,NaN,None
3,332852229,2013,7,regular,2013-10-12 23:30:00+00:00,False,Florida International,Conference USA,UAB,Conference USA,218,FIU Stadium,81.0,66.2,61.0,0.0,0,50.0,8.1,1015.9,NaN,None
4,332572229,2013,3,regular,2013-09-14 22:00:00+00:00,False,Florida International,Conference USA,Bethune-Cookman,MEAC,218,FIU Stadium,88.0,73.8,63.0,0.0,0,110.0,11.4,1013.0,NaN,None


In [5]:
# Load games data
all_dfs = []
id_cols = ['id', 'season', 'week', 'season_type', 'completed', 'neutral_site', 'venue_id']
home_cols = ['home_id', 'home_team', 'home_conference', 'home_division', 'home_points', 'home_pregame_elo']
away_cols = ['away_id', 'away_team', 'away_conference', 'away_division', 'away_points', 'away_pregame_elo']

games_dir = join(DATA_DIR, 'games')
if not os.path.exists(games_dir):
    os.makedirs(games_dir)


for year in range(2012, 2025): # Adjusted to include 2012 for rolling window
    file_path = join(games_dir, f'{year}.parquet')
    if os.path.exists(file_path):
        print(f'Reading {file_path} from cached data')
        tmp = pd.read_parquet(file_path)
    else:
        print(f'Fetching {year} games data from CFBD API')
        with cfbd.ApiClient(configuration) as api_client:
            api_instance = cfbd.GamesApi(api_client)
            data = api_instance.get_games(year=year)
        tmp = pd.DataFrame([val.to_dict() for val in data])
        tmp.columns = convert_to_snake_case(tmp.columns)
        tmp = tmp[id_cols + [
                            'home_id','home_team','home_conference','home_classification','home_points','home_pregame_elo',
                            'away_id','away_team','away_conference','away_classification','away_points','away_pregame_elo']]
        tmp['season_type'] = tmp['season_type'].apply(lambda x: x.value)
        tmp = tmp.assign(
            home_division=tmp['home_classification'].apply(lambda x: x.value if x is not None else None),
            away_division=tmp['away_classification'].apply(lambda x: x.value if x is not None else None)
        ).drop(columns=['home_classification','away_classification'])
        tmp = tmp[id_cols + home_cols + away_cols]
        tmp.to_parquet(file_path)
    all_dfs.append(tmp)
del tmp

games = pd.concat(all_dfs, ignore_index=True).reset_index(drop=True)
games.head()

Reading ../data/games/2012.parquet from cached data
Reading ../data/games/2013.parquet from cached data
Reading ../data/games/2014.parquet from cached data
Reading ../data/games/2015.parquet from cached data
Reading ../data/games/2016.parquet from cached data
Reading ../data/games/2017.parquet from cached data
Reading ../data/games/2018.parquet from cached data
Reading ../data/games/2019.parquet from cached data
Reading ../data/games/2020.parquet from cached data
Reading ../data/games/2021.parquet from cached data
Reading ../data/games/2022.parquet from cached data
Reading ../data/games/2023.parquet from cached data
Reading ../data/games/2024.parquet from cached data


,id,season,week,season_type,completed,neutral_site,venue_id,home_id,home_team,home_conference,home_division,home_points,home_pregame_elo,away_id,away_team,away_conference,away_division,away_points,away_pregame_elo,start_date
0,322432710,2012,1,regular,True,False,3746.0,2710,Western Illinois,MVFC,fcs,23.0,NaN,2086,Butler,Pioneer,fcs,15.0,NaN,NaT
1,322432117,2012,1,regular,True,False,3786.0,2117,Central Michigan,Mid-American,fbs,38.0,NaN,2546,Southeast Missouri State,OVC,fcs,27.0,NaN,NaT
2,322432050,2012,1,regular,True,False,3919.0,2050,Ball State,Mid-American,fbs,37.0,1299.0,2199,Eastern Michigan,Mid-American,fbs,26.0,1231.0,NaT
3,322432006,2012,1,regular,True,False,3768.0,2006,Akron,Mid-American,fbs,14.0,997.0,2116,UCF,Conference USA,fbs,56.0,1559.0,NaT
4,322430238,2012,1,regular,True,False,3973.0,238,Vanderbilt,SEC,fbs,13.0,1573.0,2579,South Carolina,SEC,fbs,17.0,1743.0,NaT


In [6]:
# Load plays data
all_dfs = []
cols = ['id', 'drive_id', 'game_id', 'drive_number', 'play_number', 'offense',
       'offense_conference', 'offense_score', 'defense', 'home', 'away',
       'defense_conference', 'defense_score', 'period', 'offense_timeouts',
       'defense_timeouts', 'yardline', 'yards_to_goal', 'down', 'distance',
       'yards_gained', 'scoring', 'play_type', 'play_text', 'ppa',
       'clock_minutes', 'clock_seconds']

plays_dir = join(DATA_DIR, 'plays')
if not os.path.exists(plays_dir):
    os.makedirs(plays_dir)

for year in range(2012, 2025):  # Adjusted to include 2012 for rolling window
    season_file_path = join(plays_dir, f'{year}.parquet')
    
    if os.path.exists(season_file_path):
        print(f'Reading {season_file_path} from cached data')
        season_plays = pd.read_parquet(season_file_path)
    else:
        print(f'Fetching all weeks plays data for {year} from CFBD API')
        weeks = games.query('season == @year').week.unique().tolist()

        season_dfs = []
        for week in weeks:
            print(f'Fetching {year} plays data for week {week} from CFBD API')
            with cfbd.ApiClient(configuration) as api_client:
                api_instance = cfbd.PlaysApi(api_client)
                data = api_instance.get_plays(year=year, week=week)
            tmp = pd.DataFrame([val.to_dict() for val in data])
            if tmp.empty:
                print(f'No plays data for {year} week {week}')
                continue
            tmp.columns = convert_to_snake_case(tmp.columns)

            tmp['clock_minutes'] = tmp['clock'].apply(lambda x: x['minutes'])
            tmp['clock_seconds'] = tmp['clock'].apply(lambda x: x['seconds'])
            tmp.drop(columns=['clock'], inplace=True)

            tmp = tmp[cols]
            season_dfs.append(tmp)
        
        season_plays = pd.concat(season_dfs, ignore_index=True).reset_index(drop=True)
        
        season_plays.to_parquet(season_file_path)
        print(f'Saved plays data for {year} to {season_file_path}')
        del season_dfs, tmp

    all_dfs.append(season_plays)

    del season_plays
    
plays = pd.concat(all_dfs, ignore_index=True).reset_index(drop=True)
plays.columns = plays.columns.str.replace('t_ds', 'tds', regex=False)
plays.head()

Reading ../data/plays/2012.parquet from cached data
Reading ../data/plays/2013.parquet from cached data
Reading ../data/plays/2014.parquet from cached data
Reading ../data/plays/2015.parquet from cached data
Reading ../data/plays/2016.parquet from cached data
Reading ../data/plays/2017.parquet from cached data
Reading ../data/plays/2018.parquet from cached data
Reading ../data/plays/2019.parquet from cached data
Reading ../data/plays/2020.parquet from cached data
Reading ../data/plays/2021.parquet from cached data
Reading ../data/plays/2022.parquet from cached data
Reading ../data/plays/2023.parquet from cached data
Reading ../data/plays/2024.parquet from cached data


/var/folders/cn/x384rbv51ss4zc1t6spzs7_r0000gn/T/ipykernel_80162/2545760028.py:53: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  plays = pd.concat(all_dfs, ignore_index=True).reset_index(drop=True)


,id,drive_id,game_id,drive_number,play_number,offense,offense_conference,offense_score,defense,home,away,defense_conference,defense_score,period,offense_timeouts,defense_timeouts,yardline,yards_to_goal,down,distance,yards_gained,scoring,play_type,play_text,ppa,clock_minutes,clock_seconds
0,322450221037,32245022104,322450221,4,1,Youngstown State,MVFC,7,Pittsburgh,Pittsburgh,Youngstown State,Big East,7,1,NaN,NaN,92,92,1,10,2,False,Rush,"Jamaine Cook rush for 2 yards to the YngSt 10,...",-0.207100,2,47
1,322450221048,32245022104,322450221,4,11,Youngstown State,MVFC,7,Pittsburgh,Pittsburgh,Youngstown State,Big East,7,2,NaN,NaN,36,36,2,4,0,False,Rush,"Kurt Hess rush for no gain to the Pitt 36, tac...",-0.903926,15,0
2,322450221044,32245022104,322450221,4,7,Youngstown State,MVFC,7,Pittsburgh,Pittsburgh,Youngstown State,Big East,7,2,NaN,NaN,59,59,1,10,0,False,Pass Incompletion,"Kurt Hess pass incomplete to Christian Bryan, ...",-1.138786,15,0
3,322450221045,32245022104,322450221,4,8,Youngstown State,MVFC,7,Pittsburgh,Pittsburgh,Youngstown State,Big East,7,2,NaN,NaN,59,59,2,10,5,False,Pass Completion,Kurt Hess pass complete to Marcel Caver for 5 ...,0.068045,15,0
4,322450221041,32245022104,322450221,4,5,Youngstown State,MVFC,7,Pittsburgh,Pittsburgh,Youngstown State,Big East,7,1,NaN,NaN,73,73,2,8,0,False,Pass Incompletion,"Kurt Hess pass incomplete to Christian Bryan, ...",-0.478808,2,47


In [7]:
elos = []
for year in YEARS:
    elos.append(pd.read_parquet(join(DATA_DIR, 'elo', f'{year}.parquet')))
elo = pd.concat(elos, ignore_index=True).reset_index(drop=True)
elo = elo.drop_duplicates(subset=['season', 'week', 'team_id'], keep='first')
elo.head()

,season,week,team_id,team,division,elo
0,2013,1,2,Auburn,fbs,2266.299104
2,2013,2,2,Auburn,fbs,2256.315253
3,2013,3,2,Auburn,fbs,2304.848707
4,2013,4,2,Auburn,fbs,2346.432593
5,2013,6,2,Auburn,fbs,2340.107671


In [8]:
game_team_stats_dir = join(DATA_DIR, 'game_team_stats')
if not os.path.exists(game_team_stats_dir):
    os.makedirs(game_team_stats_dir)

def flatten_team_data(row):
    game_id = row['id']
    teams_data = row['teams']
    
    # Parse the JSON string if it's not already parsed
    if isinstance(teams_data, str):
        teams_data = json.loads(teams_data)
    
    rows = []
    for team in teams_data:
        # Create base dictionary with non-stats fields
        team_row = {
            'game_id': game_id,
            'teamId': team['teamId'],
            'team': team['team'],
            'conference': team['conference'],
            'homeAway': team['homeAway'],
            'points': team['points']
        }
        
        # Add stats fields
        for stat in team['stats']:
            # Convert stat names to valid column names if needed
            col_name = stat['category']
            team_row[col_name] = stat['stat']
        
        rows.append(team_row)
    
    return rows

all_dfs = []
for year in range(2012, 2025): # Include year prior to 2013 for rolling stats
    file_path = join(DATA_DIR, 'game_team_stats', f'{year}.parquet')
    if os.path.exists(file_path):
        print(f'Reading {file_path} from cached data')
        tmp = pd.read_parquet(file_path)
        all_dfs.append(tmp)
    else:
        weeks = games.query('season == @year').week.unique().tolist()
        if not weeks:
            weeks = range(1, 16)

        season_dfs = []
        for week in weeks:
            print(f'Fetching {year} team stats data for week {week} from CFBD API')
            with cfbd.ApiClient(configuration) as api_client:
                api_instance = cfbd.GamesApi(api_client)
                data = api_instance.get_game_team_stats(year=year, week=week)
            tmp = pd.DataFrame([val.to_dict() for val in data])
            if tmp.empty:
                games_count = games.query('season == @year and week == @week and home_division == "fbs"').shape[0] + \
                    games.query('season == @year and week == @week and away_division == "fbs"').shape[0]
                print(f'No team stats data for {year} week {week}, number of FBS teams: {games_count}')
                continue

            flattened_data = []
            for _, row in tmp.iterrows():
                flattened_data.extend(flatten_team_data(row))
            result_df = pd.DataFrame(flattened_data)

            base_columns = ['game_id', 'teamId', 'team', 'conference', 'homeAway', 'points']
            stat_columns = [col for col in result_df.columns if col not in base_columns]
            result_df = result_df[base_columns + stat_columns]
            result_df.columns = convert_to_snake_case(result_df.columns)
            season_dfs.append(result_df)

        season_df = pd.concat(season_dfs, ignore_index=True).reset_index(drop=True)

        all_dfs.append(season_df)
        season_df.to_parquet(file_path)

team_stats = (
    games[['id', 'season', 'week', 'season_type']].rename(columns={'id': 'game_id'})
    .merge(
        pd.concat(all_dfs, ignore_index=True).reset_index(drop=True),
        on='game_id', how='left'
    )
    .dropna(subset=['team_id'])
    .sort_values(
        by=['season', 'season_type', 'week', 'team_id'], 
        ignore_index=True, 
        ascending=[True, False, True, True]
    )
)
team_stats.head()

Reading ../data/game_team_stats/2012.parquet from cached data
Reading ../data/game_team_stats/2013.parquet from cached data
Reading ../data/game_team_stats/2014.parquet from cached data
Reading ../data/game_team_stats/2015.parquet from cached data
Reading ../data/game_team_stats/2016.parquet from cached data
Reading ../data/game_team_stats/2017.parquet from cached data
Reading ../data/game_team_stats/2018.parquet from cached data
Reading ../data/game_team_stats/2019.parquet from cached data
Reading ../data/game_team_stats/2020.parquet from cached data
Reading ../data/game_team_stats/2021.parquet from cached data
Reading ../data/game_team_stats/2022.parquet from cached data
Reading ../data/game_team_stats/2023.parquet from cached data
Reading ../data/game_team_stats/2024.parquet from cached data


,game_id,season,week,season_type,team_id,team,conference,home_away,points,possession_time,interceptions,fumbles_lost,turnovers,total_penalties_yards,yards_per_rush_attempt,rushing_attempts,rushing_yards,yards_per_pass,completion_attempts,net_passing_yards,total_yards,fourth_down_eff,third_down_eff,first_downs,passes_intercepted,interception_tds,interception_yards,kicking_points,kick_returns,kick_return_tds,kick_return_yards,passing_tds,punt_returns,punt_return_tds,punt_return_yards,rushing_tds,fumbles_recovered,passes_deflected,qb_hurries,sacks,tackles,defensive_tds,tackles_for_loss,total_fumbles
0,322450002,2012,1,regular,2.0,Auburn,SEC,home,19.0,28:14,1,1,2,6-38,4.9,37,180,7.2,11-27,194,374,0-1,4-13,17,1,0,12,13,1,0,21,1,1,0,5,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,322450005,2012,1,regular,5.0,UAB,Conference USA,home,29.0,41:39,1,3,4,9-47,3.0,44,134,9.6,19-34,325,459,0-2,8-16,23,None,None,None,5,5,0,82,2,3,0,-6,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,322450006,2012,1,regular,6.0,South Alabama,Sun Belt,home,31.0,26:53,1,3,4,7-80,3.4,37,126,7.0,15-26,182,308,2-2,5-13,18,1,0,28,7,2,0,95,2,2,0,37,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,322450008,2012,1,regular,8.0,Arkansas,SEC,home,49.0,25:41,0,3,3,6-38,4.7,35,164,11.8,23-34,400,564,0-2,5-9,25,None,None,None,7,4,0,96,3,5,0,29,4,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,322430009,2012,1,regular,9.0,Arizona State,Pac-12,home,63.0,30:16,0,1,1,4-30,6.0,51,305,12.4,16-20,249,554,1-1,7-13,25,3,1,117,9,1,0,7,1,2,0,13,7,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
advanced_team_stats_dir = join(DATA_DIR, 'advanced_team_stats')
if not os.path.exists(advanced_team_stats_dir):
    os.makedirs(advanced_team_stats_dir)

def parse_stats(row):
    offense = row['offense']
    defense = row['defense']
    return pd.Series({
        'offense_ppa': offense.get('ppa'),
        'offense_successRate': offense.get('successRate'),
        'offense_explosiveness': offense.get('explosiveness'),
        'offense_passingPlays_ppa': offense.get('passingPlays', {}).get('ppa'),
        'offense_passingPlays_successRate': offense.get('passingPlays', {}).get('successRate'),
        'offense_passingPlays_explosiveness': offense.get('passingPlays', {}).get('explosiveness'),
        'offense_rushingPlays_ppa': offense.get('rushingPlays', {}).get('ppa'),
        'offense_rushingPlays_successRate': offense.get('rushingPlays', {}).get('successRate'),
        'offense_rushingPlays_explosiveness': offense.get('rushingPlays', {}).get('explosiveness'),
        'offense_stuffRate': offense.get('stuffRate'),
        'offense_lineYards': offense.get('lineYards'),
        'defense_ppa': defense.get('ppa'),
        'defense_successRate': defense.get('successRate'),
        'defense_explosiveness': defense.get('explosiveness'),
        'defense_passingPlays_ppa': defense.get('passingPlays', {}).get('ppa'),
        'defense_passingPlays_successRate': defense.get('passingPlays', {}).get('successRate'),
        'defense_passingPlays_explosiveness': defense.get('passingPlays', {}).get('explosiveness'),
        'offense_secondLevelYards': offense.get('secondLevelYards'),
        'offense_openFieldYards': offense.get('openFieldYards'),
        'defense_rushingPlays_ppa': defense.get('rushingPlays', {}).get('ppa'),
        'defense_rushingPlays_successRate': defense.get('rushingPlays', {}).get('successRate'),
        'defense_rushingPlays_explosiveness': defense.get('rushingPlays', {}).get('explosiveness'),
        'defense_stuffRate': defense.get('stuffRate'),
        'defense_lineYards': defense.get('lineYards'),
        'defense_secondLevelYards': defense.get('secondLevelYards'),
        'defense_openFieldYards': defense.get('openFieldYards'),
    })

all_dfs = []
for year in range(2012, 2025): # Include year prior to 2013 for rolling stats
    file_path = join(DATA_DIR, 'advanced_team_stats', f'{year}.parquet')
    if os.path.exists(file_path):
        print(f'Reading {file_path} from cached data')
        tmp = pd.read_parquet(file_path)
        all_dfs.append(tmp)
    else:
        regular_weeks = team_stats.query('season == @year and season_type=="regular"').week.unique().tolist()
        postseason_weeks = team_stats.query('season == @year and season_type=="postseason"').week.unique().tolist()

        season_dfs = []
        for season_type in ['regular', 'postseason']:
            weeks = regular_weeks if season_type == 'regular' else postseason_weeks
            for week in weeks:
                print(f'Fetching {year} team stats data for {season_type} week {week} from CFBD API')
                with cfbd.ApiClient(configuration) as api_client:
                    api_instance = cfbd.StatsApi(api_client)
                    data = api_instance.get_advanced_game_stats(
                        year=year,
                        week=week,
                        season_type=cfbd.SeasonType(season_type)
                    )
                tmp = pd.DataFrame([val.to_dict() for val in data])
                
                result_df = (
                    tmp.join(tmp.apply(parse_stats, axis=1))
                    .drop(columns=['offense', 'defense', 'opponent'])
                    .assign(season_type=season_type)
                )

                result_df.columns = convert_to_snake_case(result_df.columns)

                base_columns = ['game_id', 'season', 'team', 'season_type']
                stat_columns = [col for col in result_df.columns if col not in base_columns]
                result_df = result_df[base_columns + stat_columns]
                season_dfs.append(result_df)

            season_df = pd.concat(season_dfs, ignore_index=True).reset_index(drop=True)

            all_dfs.append(season_df)
            season_df.to_parquet(file_path)

advanced_team_stats = (
    pd.concat(all_dfs, ignore_index=True).reset_index(drop=True)
    .sort_values(
        by=['team', 'season', 'season_type', 'week'], 
        ignore_index=True, 
        ascending=[True, True, False, True]
    )
)
advanced_team_stats.head()

Reading ../data/advanced_team_stats/2012.parquet from cached data
Reading ../data/advanced_team_stats/2013.parquet from cached data
Reading ../data/advanced_team_stats/2014.parquet from cached data
Reading ../data/advanced_team_stats/2015.parquet from cached data
Reading ../data/advanced_team_stats/2016.parquet from cached data
Reading ../data/advanced_team_stats/2017.parquet from cached data
Reading ../data/advanced_team_stats/2018.parquet from cached data
Reading ../data/advanced_team_stats/2019.parquet from cached data
Reading ../data/advanced_team_stats/2020.parquet from cached data
Reading ../data/advanced_team_stats/2021.parquet from cached data
Reading ../data/advanced_team_stats/2022.parquet from cached data
Reading ../data/advanced_team_stats/2023.parquet from cached data
Reading ../data/advanced_team_stats/2024.parquet from cached data


,game_id,season,team,season_type,week,offense_ppa,offense_success_rate,offense_explosiveness,offense_passing_plays_ppa,offense_passing_plays_success_rate,offense_passing_plays_explosiveness,offense_rushing_plays_ppa,offense_rushing_plays_success_rate,offense_rushing_plays_explosiveness,offense_stuff_rate,offense_line_yards,defense_ppa,defense_success_rate,defense_explosiveness,defense_passing_plays_ppa,defense_passing_plays_success_rate,defense_passing_plays_explosiveness,offense_second_level_yards,offense_open_field_yards,defense_rushing_plays_ppa,defense_rushing_plays_success_rate,defense_rushing_plays_explosiveness,defense_stuff_rate,defense_line_yards,defense_second_level_yards,defense_open_field_yards
0,332990166,2013,Abilene Christian,regular,9,0.223948,0.442857,1.110309,0.325306,0.500000,1.196165,0.078084,0.304348,1.141636,0.130435,3.369565,0.199207,0.534247,0.845228,0.504164,0.620690,1.135052,1.173913,1.956522,0.025083,0.452381,0.719215,0.214286,3.054762,1.238095,1.142857
1,400548403,2014,Abilene Christian,regular,1,0.258485,0.461538,1.287510,0.590613,0.547619,1.721632,-0.134873,0.323529,0.619187,0.117647,2.961765,0.287588,0.518072,1.261359,0.447216,0.450980,1.953486,0.764706,0.470588,0.074896,0.612903,0.556223,0.161290,3.425806,1.483871,0.677419
2,400548413,2014,Abilene Christian,regular,3,0.367337,0.460526,1.425814,0.719482,0.527778,1.754093,0.073443,0.384615,1.161582,0.230769,2.820513,0.365256,0.424242,1.492707,0.281203,0.315789,1.973852,1.410256,1.256410,0.479327,0.571429,1.131849,0.214286,3.125000,1.178571,0.571429
3,400799923,2015,Abilene Christian,regular,1,-0.040340,0.324324,1.093651,-0.003188,0.380000,1.173526,-0.117741,0.208333,0.790128,0.208333,2.266667,0.171840,0.458824,1.156626,0.097262,0.355556,1.511928,0.541667,0.000000,0.259238,0.552632,0.978046,0.157895,3.247368,1.236842,1.684211
4,400869175,2016,Abilene Christian,regular,1,0.209173,0.301587,1.920856,0.530104,0.323529,2.711378,-0.167092,0.275862,0.833889,0.241379,2.293103,0.307413,0.481013,1.197367,1.031893,0.500000,2.860137,0.551724,0.517241,0.192573,0.469697,0.940196,0.090909,3.778788,1.454545,1.378788


In [12]:
team_stats.columns

Index(['game_id', 'season', 'week', 'season_type', 'team_id', 'team',
       'conference', 'home_away', 'points', 'possession_time', 'interceptions',
       'fumbles_lost', 'turnovers', 'total_penalties_yards',
       'yards_per_rush_attempt', 'rushing_attempts', 'rushing_yards',
       'yards_per_pass', 'completion_attempts', 'net_passing_yards',
       'total_yards', 'fourth_down_eff', 'third_down_eff', 'first_downs',
       'passes_intercepted', 'interception_tds', 'interception_yards',
       'kicking_points', 'kick_returns', 'kick_return_tds',
       'kick_return_yards', 'passing_tds', 'punt_returns', 'punt_return_tds',
       'punt_return_yards', 'rushing_tds', 'fumbles_recovered',
       'passes_deflected', 'qb_hurries', 'sacks', 'tackles', 'defensive_tds',
       'tackles_for_loss', 'total_fumbles'],
      dtype='object')

In [13]:
team_stats = (
    team_stats
    .sort_values(
        by=['team_id', 'season', 'season_type', 'week'], 
        ignore_index=True, 
        ascending=[True, True, False, True]
    )
    .assign(
        rushing_attempts=lambda x: x.rushing_attempts.astype(float),
        rushing_yards=lambda x: x.rushing_yards.astype(float),
    )
)

median_rushing_attempts = team_stats.query('rushing_attempts > 0').rushing_attempts.median(skipna=True)
median_rushing_yards = team_stats.query('rushing_attempts > 0').rushing_yards.median(skipna=True)

cols = [
    'game_id', 'season', 'week', 'season_type', 'team_id', 'team',
    'rushing_attempts', 'rushing_yards',
]

team_stats = (
    team_stats
    .assign(
        rushing_attempts=np.where(
            (team_stats.rushing_attempts > 0) & team_stats.rushing_attempts.notnull(),
            team_stats.rushing_attempts,
            median_rushing_attempts
        ),
        rushing_yards=np.where(
            (team_stats.rushing_yards > 0) & team_stats.rushing_yards.notnull(),
            team_stats.rushing_yards,
            median_rushing_yards
        ),
        kicking_points=lambda x: x.kicking_points.fillna(0).astype(float)
    )
    [cols]
    .dropna(subset=['team_id', 'team', 'season', 'week', 'game_id'])
    .reset_index(drop=True)
)
team_stats.head()

,game_id,season,week,season_type,team_id,team,rushing_attempts,rushing_yards
0,322450002,2012,1,regular,2.0,Auburn,37.0,180.0
1,322520344,2012,2,regular,2.0,Auburn,39.0,91.0
2,322590002,2012,3,regular,2.0,Auburn,42.0,255.0
3,322660002,2012,4,regular,2.0,Auburn,30.0,86.0
4,322800002,2012,6,regular,2.0,Auburn,32.0,40.0


In [24]:
advanced_stat_cols = [
    'game_id', 'season', 'team', 'season_type', 'week', 
    'offense_rushing_plays_ppa',
    'offense_rushing_plays_success_rate',
    'offense_rushing_plays_explosiveness', 
    'offense_stuff_rate',
    'offense_line_yards',
    'offense_second_level_yards',
    'offense_open_field_yards', 
    'defense_rushing_plays_ppa',
    'defense_rushing_plays_success_rate',
    'defense_rushing_plays_explosiveness', 
    'defense_stuff_rate',
    'defense_line_yards', 
    'defense_second_level_yards',
    'defense_open_field_yards'
]

team_stats_joined = (
    team_stats
    .rename(columns={
        'rushing_yards': 'offense_rushing_yards',
        'rushing_attempts': 'offense_rushing_attempts'
    })
    .merge(
        advanced_team_stats[advanced_stat_cols]
            .drop(columns=['season', 'week', 'season_type']),
        on=['game_id', 'team'],
        how='left'
    )
    .merge(
        team_stats[['game_id','team_id', 'rushing_attempts', 'rushing_yards']]
            .rename(columns={
                'team_id': 'opponent_team_id',
                'rushing_attempts': 'defense_rushing_attempts',
                'rushing_yards': 'defense_rushing_yards'
            }),
        on=['game_id'],
        how='left'
    )
    .query('team_id != opponent_team_id')
    .drop(columns=['opponent_team_id'])
    .sort_values(
        by=['team_id', 'season', 'season_type', 'week'], 
        ignore_index=False, 
        ascending=[True, True, False, True]
    )
    .dropna(subset=[
        'offense_rushing_attempts', 'offense_rushing_yards',
        'defense_rushing_attempts', 'defense_rushing_yards',
        'offense_rushing_plays_ppa',
        'offense_rushing_plays_success_rate',
        'offense_rushing_plays_explosiveness', 'offense_stuff_rate',
        'offense_line_yards', 'offense_second_level_yards',
        'offense_open_field_yards', 'defense_rushing_plays_ppa',
        'defense_rushing_plays_success_rate',
        'defense_rushing_plays_explosiveness', 'defense_stuff_rate',
        'defense_line_yards', 'defense_second_level_yards',
        'defense_open_field_yards'
    ])
)

rolling_stats_last6 = (
    team_stats_joined[[
        'team_id', 'offense_rushing_attempts', 'offense_rushing_yards',
        'defense_rushing_attempts', 'defense_rushing_yards',
        'offense_rushing_plays_ppa',
        'offense_rushing_plays_success_rate',
        'offense_rushing_plays_explosiveness', 'offense_stuff_rate',
        'offense_line_yards', 'offense_second_level_yards',
        'offense_open_field_yards', 'defense_rushing_plays_ppa',
        'defense_rushing_plays_success_rate',
        'defense_rushing_plays_explosiveness', 'defense_stuff_rate',
        'defense_line_yards', 'defense_second_level_yards',
        'defense_open_field_yards'
    ]]
    .groupby('team_id')
    .shift(1)
    .rolling(window=6, min_periods=6)
    .sum()
)

rolling_stats_last6 = (
    team_stats[['game_id', 'season', 'week', 'season_type', 'team_id', 'team']]
    .merge(
        rolling_stats_last6.rename(columns={
            'offense_rushing_attempts': 'last6_offense_rush_att',
            'offense_rushing_yards': 'last6_offense_rush_yds',
            'defense_rushing_attempts': 'last6_defense_rush_att',
            'defense_rushing_yards': 'last6_defense_rush_yds',
            'offense_rushing_plays_ppa': 'last6_offense_rushing_plays_ppa',
            'offense_rushing_plays_success_rate': 'last6_offense_rushing_plays_success_rate',
            'offense_rushing_plays_explosiveness': 'last6_offense_rushing_plays_explosiveness',
            'offense_stuff_rate': 'last6_offense_stuff_rate',
            'offense_line_yards': 'last6_offense_line_yards',
            'offense_second_level_yards': 'last6_offense_second_level_yards',
            'offense_open_field_yards': 'last6_offense_open_field_yards',
            'defense_rushing_plays_ppa': 'last6_defense_rushing_plays_ppa',
            'defense_rushing_plays_success_rate': 'last6_defense_rushing_plays_success_rate',
            'defense_rushing_plays_explosiveness': 'last6_defense_rushing_plays_explosiveness',
            'defense_stuff_rate': 'last6_defense_stuff_rate',
            'defense_line_yards': 'last6_defense_line_yards',
            'defense_second_level_yards': 'last6_defense_second_level_yards',
            'defense_open_field_yards': 'last6_defense_open_field_yards'
        }),
        left_index=True,
        right_index=True,
        how='left'
    )
    .dropna(subset=['last6_offense_rush_att'])
    .query('season >= 2013')
    .reset_index(drop=True)
    .assign(
        last6_offense_yards_per_rush=lambda x: round(x.last6_offense_rush_yds / x.last6_offense_rush_att, 1),
        last6_offense_rushing_plays_ppa=lambda x: round(x.last6_offense_rushing_plays_ppa / 6, 3),
        last6_offense_rushing_plays_success_rate=lambda x: round(x.last6_offense_rushing_plays_success_rate / 6, 3),
        last6_offense_rushing_plays_explosiveness=lambda x: round(x.last6_offense_rushing_plays_explosiveness / 6, 3),
        last6_offense_stuff_rate=lambda x: round(x.last6_offense_stuff_rate / 6, 3),
        last6_offense_line_yards=lambda x: round(x.last6_offense_line_yards / 6, 3),
        last6_offense_second_level_yards=lambda x: round(x.last6_offense_second_level_yards / 6, 3),
        last6_offense_open_field_yards=lambda x: round(x.last6_offense_open_field_yards / 6, 3),
        last6_defense_yards_per_rush=lambda x: round(x.last6_defense_rush_yds / x.last6_defense_rush_att, 1),
        last6_defense_rushing_plays_ppa=lambda x: round(x.last6_defense_rushing_plays_ppa / 6, 3),
        last6_defense_rushing_plays_success_rate=lambda x: round(x.last6_defense_rushing_plays_success_rate / 6, 3),
        last6_defense_rushing_plays_explosiveness=lambda x: round(x.last6_defense_rushing_plays_explosiveness / 6, 3),
        last6_defense_stuff_rate=lambda x: round(x.last6_defense_stuff_rate / 6, 3),
        last6_defense_line_yards=lambda x: round(x.last6_defense_line_yards / 6, 3),
        last6_defense_second_level_yards=lambda x: round(x.last6_defense_second_level_yards / 6, 3),
        last6_defense_open_field_yards=lambda x: round(x.last6_defense_open_field_yards / 6, 3)
    )
)

del team_stats_joined

rolling_stats_last6.head()

,game_id,season,week,season_type,team_id,team,last6_offense_rush_att,last6_offense_rush_yds,last6_defense_rush_att,last6_defense_rush_yds,last6_offense_rushing_plays_ppa,last6_offense_rushing_plays_success_rate,last6_offense_rushing_plays_explosiveness,last6_offense_stuff_rate,last6_offense_line_yards,last6_offense_second_level_yards,last6_offense_open_field_yards,last6_defense_rushing_plays_ppa,last6_defense_rushing_plays_success_rate,last6_defense_rushing_plays_explosiveness,last6_defense_stuff_rate,last6_defense_line_yards,last6_defense_second_level_yards,last6_defense_open_field_yards,last6_offense_yards_per_rush,last6_defense_yards_per_rush
0,332500002,2013,2,regular,2.0,Auburn,221.0,753.0,245.0,1149.0,0.042,0.405,0.885,0.263,2.506,1.274,0.950,0.034,0.408,0.718,0.172,3.027,1.058,1.223,3.4,4.7
1,332640099,2013,4,regular,2.0,Auburn,217.0,676.0,238.0,1038.0,0.024,0.398,0.882,0.277,2.391,1.233,0.937,-0.027,0.383,0.706,0.194,2.876,0.966,1.040,3.1,4.4
2,332850002,2013,7,regular,2.0,Auburn,212.0,709.0,250.0,1224.0,0.077,0.446,0.881,0.259,2.782,1.427,0.962,0.048,0.403,0.756,0.183,3.012,1.099,1.210,3.3,4.9
3,332990002,2013,9,regular,2.0,Auburn,215.0,765.0,241.0,1142.0,0.057,0.438,0.865,0.257,2.814,1.466,0.993,0.011,0.406,0.751,0.187,2.981,1.085,1.269,3.6,4.7
4,333132633,2013,11,regular,2.0,Auburn,212.0,736.0,239.0,1249.0,0.020,0.403,0.794,0.215,2.909,1.335,0.833,0.065,0.428,0.832,0.197,3.031,1.195,1.394,3.5,5.2


In [28]:
advanced_stat_cols = [
    'game_id', 'season', 'team', 'season_type', 'week', 
    'offense_rushing_plays_ppa',
    'offense_rushing_plays_success_rate',
    'offense_rushing_plays_explosiveness', 
    'offense_stuff_rate',
    'offense_line_yards',
    'offense_second_level_yards',
    'offense_open_field_yards', 
    'defense_rushing_plays_ppa',
    'defense_rushing_plays_success_rate',
    'defense_rushing_plays_explosiveness', 
    'defense_stuff_rate',
    'defense_line_yards', 
    'defense_second_level_yards',
    'defense_open_field_yards'
]

team_stats_joined = (
    team_stats
    .rename(columns={
        'rushing_yards': 'offense_rushing_yards',
        'rushing_attempts': 'offense_rushing_attempts'
    })
    .merge(
        advanced_team_stats[advanced_stat_cols]
            .drop(columns=['season', 'week', 'season_type']),
        on=['game_id', 'team'],
        how='left'
    )
    .merge(
        team_stats[['game_id','team_id', 'rushing_attempts', 'rushing_yards']]
            .rename(columns={
                'team_id': 'opponent_team_id',
                'rushing_attempts': 'defense_rushing_attempts',
                'rushing_yards': 'defense_rushing_yards'
            }),
        on=['game_id'],
        how='left'
    )
    .query('team_id != opponent_team_id')
    .drop(columns=['opponent_team_id'])
    .sort_values(
        by=['team_id', 'season', 'season_type', 'week'], 
        ignore_index=False, 
        ascending=[True, True, False, True]
    )
    .dropna(subset=[
        'offense_rushing_attempts', 'offense_rushing_yards',
        'defense_rushing_attempts', 'defense_rushing_yards',
        'offense_rushing_plays_ppa',
        'offense_rushing_plays_success_rate',
        'offense_rushing_plays_explosiveness', 'offense_stuff_rate',
        'offense_line_yards', 'offense_second_level_yards',
        'offense_open_field_yards', 'defense_rushing_plays_ppa',
        'defense_rushing_plays_success_rate',
        'defense_rushing_plays_explosiveness', 'defense_stuff_rate',
        'defense_line_yards', 'defense_second_level_yards',
        'defense_open_field_yards'
    ])
)

rolling_stats_last12 = (
    team_stats_joined[[
        'team_id', 'offense_rushing_attempts', 'offense_rushing_yards',
        'defense_rushing_attempts', 'defense_rushing_yards',
        'offense_rushing_plays_ppa',
        'offense_rushing_plays_success_rate',
        'offense_rushing_plays_explosiveness', 'offense_stuff_rate',
        'offense_line_yards', 'offense_second_level_yards',
        'offense_open_field_yards', 'defense_rushing_plays_ppa',
        'defense_rushing_plays_success_rate',
        'defense_rushing_plays_explosiveness', 'defense_stuff_rate',
        'defense_line_yards', 'defense_second_level_yards',
        'defense_open_field_yards'
    ]]
    .groupby('team_id')
    .shift(1)
    .rolling(window=12, min_periods=12)
    .sum()
)

rolling_stats_last12 = (
    team_stats[['game_id', 'season', 'week', 'season_type', 'team_id', 'team']]
    .merge(
        rolling_stats_last12.rename(columns={
            'offense_rushing_attempts': 'last12_offense_rush_att',
            'offense_rushing_yards': 'last12_offense_rush_yds',
            'defense_rushing_attempts': 'last12_defense_rush_att',
            'defense_rushing_yards': 'last12_defense_rush_yds',
            'offense_rushing_plays_ppa': 'last12_offense_rushing_plays_ppa',
            'offense_rushing_plays_success_rate': 'last12_offense_rushing_plays_success_rate',
            'offense_rushing_plays_explosiveness': 'last12_offense_rushing_plays_explosiveness',
            'offense_stuff_rate': 'last12_offense_stuff_rate',
            'offense_line_yards': 'last12_offense_line_yards',
            'offense_second_level_yards': 'last12_offense_second_level_yards',
            'offense_open_field_yards': 'last12_offense_open_field_yards',
            'defense_rushing_plays_ppa': 'last12_defense_rushing_plays_ppa',
            'defense_rushing_plays_success_rate': 'last12_defense_rushing_plays_success_rate',
            'defense_rushing_plays_explosiveness': 'last12_defense_rushing_plays_explosiveness',
            'defense_stuff_rate': 'last12_defense_stuff_rate',
            'defense_line_yards': 'last12_defense_line_yards',
            'defense_second_level_yards': 'last12_defense_second_level_yards',
            'defense_open_field_yards': 'last12_defense_open_field_yards'
        }),
        left_index=True,
        right_index=True,
        how='left'
    )
    .dropna(subset=['last12_offense_rush_att'])
    .query('season >= 2013')
    .reset_index(drop=True)
    .assign(
        last12_offense_yards_per_rush=lambda x: round(x.last12_offense_rush_yds / x.last12_offense_rush_att, 1),
        last12_offense_rushing_plays_ppa=lambda x: round(x.last12_offense_rushing_plays_ppa / 12, 3),
        last12_offense_rushing_plays_success_rate=lambda x: round(x.last12_offense_rushing_plays_success_rate / 12, 3),
        last12_offense_rushing_plays_explosiveness=lambda x: round(x.last12_offense_rushing_plays_explosiveness / 12, 3),
        last12_offense_stuff_rate=lambda x: round(x.last12_offense_stuff_rate / 12, 3),
        last12_offense_line_yards=lambda x: round(x.last12_offense_line_yards / 12, 3),
        last12_offense_second_level_yards=lambda x: round(x.last12_offense_second_level_yards / 12, 3),
        last12_offense_open_field_yards=lambda x: round(x.last12_offense_open_field_yards / 12, 3),
        last12_defense_yards_per_rush=lambda x: round(x.last12_defense_rush_yds / x.last12_defense_rush_att, 1),
        last12_defense_rushing_plays_ppa=lambda x: round(x.last12_defense_rushing_plays_ppa / 12, 3),
        last12_defense_rushing_plays_success_rate=lambda x: round(x.last12_defense_rushing_plays_success_rate / 12, 3),
        last12_defense_rushing_plays_explosiveness=lambda x: round(x.last12_defense_rushing_plays_explosiveness / 12, 3),
        last12_defense_stuff_rate=lambda x: round(x.last12_defense_stuff_rate / 12, 3),
        last12_defense_line_yards=lambda x: round(x.last12_defense_line_yards / 12, 3),
        last12_defense_second_level_yards=lambda x: round(x.last12_defense_second_level_yards / 12, 3),
        last12_defense_open_field_yards=lambda x: round(x.last12_defense_open_field_yards / 12, 3)
    )
)

del team_stats_joined

rolling_stats_last12.head()

,game_id,season,week,season_type,team_id,team,last12_offense_rush_att,last12_offense_rush_yds,last12_defense_rush_att,last12_defense_rush_yds,last12_offense_rushing_plays_ppa,last12_offense_rushing_plays_success_rate,last12_offense_rushing_plays_explosiveness,last12_offense_stuff_rate,last12_offense_line_yards,last12_offense_second_level_yards,last12_offense_open_field_yards,last12_defense_rushing_plays_ppa,last12_defense_rushing_plays_success_rate,last12_defense_rushing_plays_explosiveness,last12_defense_stuff_rate,last12_defense_line_yards,last12_defense_second_level_yards,last12_defense_open_field_yards,last12_offense_yards_per_rush,last12_defense_yards_per_rush
0,340060002,2013,1,postseason,2.0,Auburn,438.0,1781.0,484.0,2371.0,0.040,0.399,0.817,0.218,2.855,1.259,0.809,0.049,0.430,0.777,0.185,3.025,1.052,1.350,4.1,4.9
1,400548186,2014,2,regular,2.0,Auburn,447.0,1896.0,455.0,2171.0,0.039,0.400,0.802,0.212,2.857,1.233,0.761,0.048,0.440,0.790,0.195,3.012,1.116,1.274,4.2,4.8
2,400548035,2014,5,regular,2.0,Auburn,458.0,2106.0,457.0,2143.0,0.051,0.414,0.801,0.210,2.972,1.289,0.847,0.044,0.435,0.817,0.207,2.940,1.132,1.331,4.6,4.7
3,400548369,2014,7,regular,2.0,Auburn,452.0,1971.0,456.0,2180.0,0.028,0.398,0.793,0.205,2.962,1.281,0.729,0.056,0.442,0.842,0.205,2.963,1.178,1.377,4.4,4.8
4,400548349,2014,10,regular,2.0,Auburn,474.0,2098.0,456.0,2226.0,0.033,0.413,0.751,0.188,3.062,1.250,0.661,0.048,0.436,0.886,0.226,2.907,1.194,1.385,4.4,4.9


# Data Engineering

In [ ]:
rush_attempts = (
    plays    
    .assign(play_text=plays.play_text.fillna('')
        .str.replace(',', '', regex=False)
        .str.replace('.', '', regex=False)
        .str.lower())
    .assign(play_text=lambda x: ' ' + x.play_text + ' ')
)
sure_run_plays = [
    'Rush', 'Rushing Touchdown'
]
potential_run_plays = [
    'Safety', 'Fumble Recovery (Own)', 'Fumble Return Touchdown', 
    'Fumble Recovery (Opponent)', 'Uncategorized', 'placeholder'
]
run_condition = (
    rush_attempts['play_type'].isin(sure_run_plays) |
    (
        rush_attempts['play_type'].isin(potential_run_plays) &
        ~rush_attempts['play_text'].str.contains(" pass ") &
        ~rush_attempts['play_text'].str.contains(" sacked ") &
        ~rush_attempts['play_text'].str.contains(" sack ") &
        ~rush_attempts['play_text'].str.contains(" interception ") &
        ~rush_attempts['play_text'].str.contains(" intercepted ") &
        ~rush_attempts['play_text'].str.contains(" kneel ") &
        ~rush_attempts['play_text'].str.contains(" kneeling ") &
        ~rush_attempts['play_text'].str.contains(" kneels ") &
        (
            rush_attempts['play_text'].str.contains(" run ") |
            rush_attempts['play_text'].str.contains(" rush ") |
            rush_attempts['play_text'].str.contains(" rushing ") |
            rush_attempts['play_text'].str.contains(" rushed ")
        )
    )
)
rush_attempts = (
    rush_attempts[run_condition]
    # dropping all penalty plays, difficult to parse penalty outcomes
    .query('~play_text.str.contains(" penalty ")')
    .merge(
        games[['id', 'season']].rename(columns={'id': 'game_id'}),
        on='game_id',
        how='left'
    )
    .query('season >= 2013')
    # Filter out kneel plays
    .query('~play_text.str.contains(" kneel ") & ~play_text.str.contains(" kneeling ") & '
           '~play_text.str.contains(" kneels ") & ~play_text.str.contains(" knee ")')
    .query('~play_text.str.containts(" punt ")')
    .drop(columns=['season'])
    .query(
        'down.isin([1,2,3,4]) and '
        'period.isin([1, 2, 3, 4])'
    )
    .assign(
        fumble = lambda x: np.where(
            x.play_text.str.contains(" fumble ") |
            x.play_text.str.contains(" fumbled "),
            1, 0
        ),
    )
    .reset_index(drop=True)
)

In [ ]:
rush_attempts = (
    rush_attempts
    .assign(
        rush_yards = lambda x: np.where(
            x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled)(?: draw play)? for (-?\d+) (?:yard|yd|yds|yards)')[0].notnull(),
            x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled)(?: draw play)? for (-?\d+) (?:yard|yd|yds|yards)')[0].astype(float),
            np.where(
                x.play_text.str.contains(' (?:rush|run|rushed|scrambles|scrambled) for no gain ') | 
                x.play_text.str.contains(' (?:rush|run|rushed|scrambles|scrambled) up the middle for no gain ') |
                x.play_text.str.contains(' (?:rush|run|rushed|scrambles|scrambled) left for no gain ') |
                x.play_text.str.contains(' (?:rush|run|rushed|scrambles|scrambled) right for no gain ') |
                x.play_text.str.contains(' (?:rush|run|rushed|scrambles|scrambled) over right guard for no gain ') |
                x.play_text.str.contains(' (?:rush|run|rushed|scrambles|scrambled) over left guard for no gain ') |
                x.play_text.str.contains(' (?:rush|run|rushed|scrambles|scrambled) over right tackle for no gain ') |
                x.play_text.str.contains(' (?:rush|run|rushed|scrambles|scrambled) over left tackle for no gain ') |
                x.play_text.str.contains(' (?:rush|run|rushed|scrambles|scrambled) over right end for no gain ') |
                x.play_text.str.contains(' (?:rush|run|rushed|scrambles|scrambled) over left end for no gain ') |
                x.play_text.str.contains(' (?:rush|run|rushed|scrambles|scrambled) draw play for no gain ') |
                x.play_text.str.contains(' (?:rush|run|rushed|scrambles|scrambled) quarterback draw for no gain ') |
                x.play_text.str.contains(' (?:rush|run|rushed|scrambles|scrambled) reverse for no gain ') |
                x.play_text.str.contains(' (?:rush|run|rushed|scrambles|scrambled) quarterback sneak for no gain ') |
                x.play_text.str.contains(' (?:rush|run|rushed|scrambles|scrambled) middle for no gain ') |
                x.play_text.str.contains(r' (?:rush|run|rushed|scrambles|scrambled) to \w+ for no gain ') |
                (x.play_text.str.contains(' no gain ') & ~x.play_text.str.contains(' fumble')),
                0.0,
                np.where(
                    x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled)(?: draw play)? for a loss of (\d+) (?:yard|yd|yds|yards)')[0].notnull(),
                    -x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled)(?: draw play)? for a loss of (\d+) (?:yard|yd|yds|yards)')[0].astype(float),
                    np.where(
                        x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) up (?:the )?middle for (-?\d+) (?:yard|yd|yds|yards)')[0].notnull(),
                        x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) up (?:the )?middle for (-?\d+) (?:yard|yd|yds|yards)')[0].astype(float),
                        np.where(
                            x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) over (?:the )?right guard for (-?\d+) (?:yard|yd|yds|yards)')[0].notnull(),
                            x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) over (?:the )?right guard for (-?\d+) (?:yard|yd|yds|yards)')[0].astype(float),
                            np.where(
                                x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) over (?:the )?left guard for (-?\d+) (?:yard|yd|yds|yards)')[0].notnull(),
                                x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) over (?:the )?left guard for (-?\d+) (?:yard|yd|yds|yards)')[0].astype(float),
                                np.where(
                                    x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) over (?:the )?right tackle for (-?\d+) (?:yard|yd|yds|yards)')[0].notnull(),
                                    x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) over (?:the )?right tackle for (-?\d+) (?:yard|yd|yds|yards)')[0].astype(float),
                                    np.where(
                                        x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) over (?:the )?left tackle for (-?\d+) (?:yard|yd|yds|yards)')[0].notnull(),
                                        x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) over (?:the )?left tackle for (-?\d+) (?:yard|yd|yds|yards)')[0].astype(float),
                                        np.where(
                                            x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) over (?:the )?right end for (-?\d+) (?:yard|yd|yds|yards)')[0].notnull(),
                                            x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) over (?:the )?right end for (-?\d+) (?:yard|yd|yds|yards)')[0].astype(float),
                                            np.where(
                                                x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) over (?:the )?left end for (-?\d+) (?:yard|yd|yds|yards)')[0].notnull(),
                                                x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) over (?:the )?left end for (-?\d+) (?:yard|yd|yds|yards)')[0].astype(float),
                                                np.where(
                                                    x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) (?:to the )?right for (-?\d+) (?:yard|yd|yds|yards)')[0].notnull(),
                                                    x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) (?:to the )?right for (-?\d+) (?:yard|yd|yds|yards)')[0].astype(float),
                                                    np.where(
                                                        x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) (?:to the )?left for (-?\d+) (?:yard|yd|yds|yards)')[0].notnull(),
                                                        x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) (?:to the )?left for (-?\d+) (?:yard|yd|yds|yards)')[0].astype(float),
                                                        np.where(
                                                            x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) up (?:the )?middle for a loss of (\d+) (?:yard|yd|yds|yards)')[0].notnull(),
                                                            -x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) up (?:the )?middle for a loss of (\d+) (?:yard|yd|yds|yards)')[0].astype(float),
                                                            np.where(
                                                                x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) over (?:the )?right guard for a loss of (\d+) (?:yard|yd|yds|yards)')[0].notnull(),
                                                                -x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) over (?:the )?right guard for a loss of (\d+) (?:yard|yd|yds|yards)')[0].astype(float),
                                                                np.where(
                                                                    x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) over (?:the )?left guard for a loss of (\d+) (?:yard|yd|yds|yards)')[0].notnull(),
                                                                    -x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) over (?:the )?left guard for a loss of (\d+) (?:yard|yd|yds|yards)')[0].astype(float),
                                                                    np.where(
                                                                        x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) over (?:the )?right tackle for a loss of (\d+) (?:yard|yd|yds|yards)')[0].notnull(),
                                                                        -x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) over (?:the )?right tackle for a loss of (\d+) (?:yard|yd|yds|yards)')[0].astype(float),
                                                                        np.where(
                                                                            x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) over (?:the )?left tackle for a loss of (\d+) (?:yard|yd|yds|yards)')[0].notnull(),
                                                                            -x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) over (?:the )?left tackle for a loss of (\d+) (?:yard|yd|yds|yards)')[0].astype(float),
                                                                            np.where(
                                                                                x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) over (?:the )?right end for a loss of (\d+) (?:yard|yd|yds|yards)')[0].notnull(),
                                                                                -x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) over (?:the )?right end for a loss of (\d+) (?:yard|yd|yds|yards)')[0].astype(float),
                                                                                np.where(
                                                                                    x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) over (?:the )?left end for a loss of (\d+) (?:yard|yd|yds|yards)')[0].notnull(),
                                                                                    -x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) over (?:the )?left end for a loss of (\d+) (?:yard|yd|yds|yards)')[0].astype(float),
                                                                                    np.where(
                                                                                        x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) (?:to the )?right for a loss of (\d+) (?:yard|yd|yds|yards)')[0].notnull(),
                                                                                        -x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) (?:to the )?right for a loss of (\d+) (?:yard|yd|yds|yards)')[0].astype(float),
                                                                                        np.where(
                                                                                            x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) (?:to the )?left for a loss of (\d+) (?:yard|yd|yds|yards)')[0].notnull(),
                                                                                            -x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) (?:to the )?left for a loss of (\d+) (?:yard|yd|yds|yards)')[0].astype(float),
                                                                                            np.where(
                                                                                                x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) for (-?\d+) yd')[0].notnull(),
                                                                                                x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) for (-?\d+) yd')[0].astype(float),
                                                                                                np.where(
                                                                                                    x.play_text.str.extract(r' (-?\d+) (?:yard|yd) (?:rush|run|rushed|scrambles|scrambled)')[0].notnull(),
                                                                                                    x.play_text.str.extract(r' (-?\d+) (?:yard|yd) (?:rush|run|rushed|scrambles|scrambled)')[0].astype(float),
                                                                                                    np.where(
                                                                                                        x.play_text.str.extract(r'\b(?:rush|run|rushed|scrambles|scrambled)\b to \w+ \d+ for (-?\d+) (?:yard|yd|yds|yards)')[0].notnull(),
                                                                                                        x.play_text.str.extract(r'\b(?:rush|run|rushed|scrambles|scrambled)\b to \w+ \d+ for (-?\d+) (?:yard|yd|yds|yards)')[0].astype(float),
                                                                                                        np.where(
                                                                                                            x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) (?:quarterback )?sneak for (-?\d+) (?:yard|yd|yds|yards)')[0].notnull(),
                                                                                                            x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) (?:quarterback )?sneak for (-?\d+) (?:yard|yd|yds|yards)')[0].astype(float),
                                                                                                            np.where(
                                                                                                                x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) (?:quarterback )?draw for (-?\d+) (?:yard|yd|yds|yards)')[0].notnull(),
                                                                                                                x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) (?:quarterback )?draw for (-?\d+) (?:yard|yd|yds|yards)')[0].astype(float),
                                                                                                                np.where(
                                                                                                                    x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) (?:quarterback )?sneak for a loss of (\d+) (?:yard|yd|yds|yards)')[0].notnull(),
                                                                                                                    -x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) (?:quarterback )?sneak for a loss of (\d+) (?:yard|yd|yds|yards)')[0].astype(float),
                                                                                                                    np.where(
                                                                                                                        x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) (?:quarterback )?draw for a loss of (\d+) (?:yard|yd|yds|yards)')[0].notnull(),
                                                                                                                        -x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) (?:quarterback )?draw for a loss of (\d+) (?:yard|yd|yds|yards)')[0].astype(float),
                                                                                                                        np.where(
                                                                                                                            x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) to \w+ for (-?\d+) (?:yard|yd|yards)')[0].notnull(),
                                                                                                                            x.play_text.str.extract(r' (?:rush|run|rushed|scrambles|scrambled) to \w+ for (-?\d+) (?:yard|yd|yards)')[0].astype(float),
                                                                                                                            np.where(
                                                                                                                                ~x.play_text.str.contains(' fumble ') & ~x.play_text.str.contains(' fumbled '),
                                                                                                                                x.yards_gained,
                                                                                                                                np.where(
                                                                                                                                    # under 200 occurrences here of over 800k plays
                                                                                                                                    x.play_text.str.contains(' fumble ') | x.play_text.str.contains(' fumbled '),
                                                                                                                                    0.0,
                                                                                                                                    np.nan
                                                                                                                                )
                                                                                                                            )
                                                                                                                        )
                                                                                                                    )
                                                                                                                )
                                                                                                            )
                                                                                                        )
                                                                                                    )
                                                                                                )
                                                                                            )
                                                                                        )
                                                                                    )
                                                                                )
                                                                            )
                                                                        )
                                                                    )
                                                                )
                                                            )
                                                        )                                   
                                                    )
                                                )
                                            )
                                        )
                                    )
                                )
                            )
                        )
                    )
                )
            )
        ),
    )
)

In [261]:
rush_attempts = (
    rush_attempts
    .assign(
        fumble_recovered_by=lambda x: np.where(
            x.fumble == 1,
            np.where(
                x.play_type.isin(['Safety', 'Fumble Recovery (Own)', 'Rushing Touchdown',]),
                "offense",
                np.where(
                    x.play_type.isin(['Fumble Recovery (Opponent)', 'Fumble Return Touchdown']),
                    "defense",
                    "unknown"
                )
            ),
            "no-fumble"
        ),
    )
    .assign(
        sack_fumble_recovery_yards = lambda x: np.where(
            (x.fumble == 1) & (x.fumble_recovered_by.isin(['offense', 'defense'])),
            np.where(
                x.play_type.isin(["Safety", "Fumble Return Touchdown"]),
                100 - x.yards_to_goal,
                np.where(
                    x.play_type.isin(["Rushing Touchdown"]),
                    x.yards_to_goal,
                    np.where(
                        x.play_text.str.extract(r' return for (\d+) yard')[0].notnull(),
                        x.play_text.str.extract(r' return for (\d+) yard')[0].astype(float),
                        np.where(
                            x.play_text.str.extract(r' return for (\d+) yd')[0].notnull(),
                            x.play_text.str.extract(r' return for (\d+) yd')[0].astype(float),
                            np.where(
                                x.play_text.str.extract(r' (\d+) yd fumble return')[0].notnull(),
                                x.play_text.str.extract(r' (\d+) yd fumble return')[0].astype(float),
                                np.where(
                                    x.play_text.str.extract(r' (\d+) yard fumble return')[0].notnull(),
                                    x.play_text.str.extract(r' (\d+) yard fumble return')[0].astype(float),
                                    np.where(
                                        x.play_text.str.extract(r' return for -(\d+) yard')[0].notnull(),
                                        -x.play_text.str.extract(r' return for -(\d+) yard')[0].astype(float),
                                        np.where(
                                            x.play_text.str.extract(r' return for -(\d+) yds ')[0].notnull(),
                                            -x.play_text.str.extract(r' return for -(\d+) yds ')[0].astype(float),
                                            np.where(
                                                x.play_text.str.extract(r' returned (\d+) yards ')[0].notnull(),
                                                x.play_text.str.extract(r' returned (\d+) yards ')[0].astype(float),
                                                np.where(
                                                    x.play_text.str.extract(r' returned (\d+) yds ')[0].notnull(),
                                                    x.play_text.str.extract(r' returned (\d+) yds ')[0].astype(float),
                                                    np.where(
                                                        x.play_text.str.extract(r' return (\d+) yard')[0].notnull(),
                                                        x.play_text.str.extract(r' return (\d+) yard')[0].astype(float),
                                                        np.where(
                                                            x.play_text.str.extract(r' return (\d+) yd')[0].notnull(),
                                                            x.play_text.str.extract(r' return (\d+) yd')[0].astype(float),
                                                            np.where(
                                                                x.play_text.str.extract(r' return for a loss of (\d+) (?:yard|yd|yds|yards)')[0].notnull(),
                                                                -x.play_text.str.extract(r' return for a loss of (\d+) (?:yard|yd|yds|yards)')[0].astype(float),
                                                                np.where(
                                                                    x.play_text.str.contains(' return for no gain ') |
                                                                    x.play_text.str.contains(' returned for no gain '),
                                                                    0.0,
                                                                    np.where(
                                                                        x.play_text.str.extract(r' returned (-?\d+) (?:yard|yd|yds|yards)')[0].notnull(),
                                                                        x.play_text.str.extract(r' returned (-?\d+) (?:yard|yd|yds|yards)')[0].astype(float),
                                                                        np.where(
                                                                            x.play_text.str.extract(r' return for loss of (\d+) (?:yard|yd|yds|yards)')[0].notnull(),
                                                                            -x.play_text.str.extract(r' return for loss of (\d+) (?:yard|yd|yds|yards)')[0].astype(float),
                                                                            0.0
                                                                        )
                                                                    )
                                                                )
                                                            )
                                                        )
                                                    )
                                                )
                                            )
                                        )
                                    )
                                )
                            )
                        )
                    )
                )
            ),
            np.nan
        )
    )
)